In [1]:
import sqlite3 as sql

In [2]:
#make sure that the connection is within the same directory as the database
conn = sql.connect("Datawarehouse_3CSC.db")

In [3]:
cursor = conn.cursor()

In [ ]:
# cursor.execute("INSERT INTO students_csc VALUES(2019130134, 'John Angelo', 'Algarne', 20, 4.20, 'M')")
cursor.execute("SELECT * FROM students_csc")

In [4]:
cursor.execute('CREATE TABLE table1(ID INTEGER PRIMARY KEY NOT NULL, NAME TEXT NOT NULL, AGE INTEGER NOT NULL, ADDRESS CHAR(50) NOT NULL, SALARY REAL NOT NULL)')

In [6]:
cursor.execute('CREATE TABLE table2(ID INTEGER PRIMARY KEY NOT NULL, DEPARTMENT CHAR(50) NOT NULL, EMP_ID INTEGER NOT NULL)')

In [19]:
conn.commit()

In [20]:
import pandas as pd

In [9]:
t1 = pd.read_csv('dept_co - table1.csv')
t1

,ID,NAME,AGE,ADDRESS,SALARY
0,1,Paul,32,California,20000
1,2,Allen,25,Texas,15000
2,3,Teddy,23,Norway,20000
3,4,Mark,25,Rich-Mond,65000
4,5,David,27,Texas,85000
5,6,Kim,22,South-Hall,45000
6,7,James,24,Houston,10000


In [10]:
t2  = pd.read_csv('dept_co - table2.csv')
t2

,ID,DEPARTMENT,EMP_ID
0,1,IT Billing,1
1,2,Engineering,2
2,3,Finance,7


In [11]:
# if_exists{‘fail’, ‘replace’, ‘append’}, default ‘fail’
t1.to_sql(name = 'table1', con = conn, index = False, if_exists = 'replace')

In [12]:
t2.to_sql(name = 'table2', con = conn, index = False, if_exists = 'replace')

In [16]:
cursor.execute("SELECT EMP_ID, NAME, DEPARTMENT FROM table1 CROSS JOIN table2")

In [17]:
cursor.fetchall()

[(1, 'Paul', 'IT Billing'),
 (2, 'Paul', 'Engineering'),
 (7, 'Paul', 'Finance'),
 (1, 'Allen', 'IT Billing'),
 (2, 'Allen', 'Engineering'),
 (7, 'Allen', 'Finance'),
 (1, 'Teddy', 'IT Billing'),
 (2, 'Teddy', 'Engineering'),
 (7, 'Teddy', 'Finance'),
 (1, 'Mark', 'IT Billing'),
 (2, 'Mark', 'Engineering'),
 (7, 'Mark', 'Finance'),
 (1, 'David', 'IT Billing'),
 (2, 'David', 'Engineering'),
 (7, 'David', 'Finance'),
 (1, 'Kim', 'IT Billing'),
 (2, 'Kim', 'Engineering'),
 (7, 'Kim', 'Finance'),
 (1, 'James', 'IT Billing'),
 (2, 'James', 'Engineering'),
 (7, 'James', 'Finance')]

In [21]:
# allows us to make a data frame
cross_join = pd.read_sql_query("SELECT EMP_ID, NAME, DEPARTMENT FROM table1 CROSS JOIN table2", conn)
cross_join

,EMP_ID,NAME,DEPARTMENT
0,1,Paul,IT Billing
1,2,Paul,Engineering
2,7,Paul,Finance
3,1,Allen,IT Billing
4,2,Allen,Engineering
5,7,Allen,Finance
6,1,Teddy,IT Billing
7,2,Teddy,Engineering
8,7,Teddy,Finance
9,1,Mark,IT Billing


In [28]:
# requires to have an inner join and on condition to filter down the dataframe
inner_join = pd.read_sql_query("SELECT EMP_ID, NAME, DEPARTMENT FROM table1 INNER JOIN table2 ON table1.ID = table2.EMP_ID", conn)
inner_join

# Natural join does not have a condition unlike inner join
# Natural join requires the same column between two tables
# Condition of natural join is somewhat weird

,EMP_ID,NAME,DEPARTMENT
0,1,Paul,IT Billing
1,2,Allen,Engineering
2,7,James,Finance


In [30]:
natural_join = pd.read_sql_query("SELECT EMP_ID, NAME, DEPARTMENT FROM table1 NATURAL JOIN table2", conn)
natural_join

,EMP_ID,NAME,DEPARTMENT
0,1,Paul,IT Billing
1,2,Allen,Engineering
2,7,Teddy,Finance


In [33]:
outer_join_t1_to_t2 = pd.read_sql_query("SELECT EMP_ID, NAME, DEPARTMENT FROM table1 LEFT OUTER JOIN table2 ON table1.ID = table2.EMP_ID", conn)
outer_join_t1_to_t2

,EMP_ID,NAME,DEPARTMENT
0,1.0,Paul,IT Billing
1,2.0,Allen,Engineering
2,NaN,Teddy,None
3,NaN,Mark,None
4,NaN,David,None
5,NaN,Kim,None
6,7.0,James,Finance


In [34]:
outer_join_t2_to_t1 = pd.read_sql_query("SELECT EMP_ID, NAME, DEPARTMENT FROM table2 LEFT OUTER JOIN table1 ON table1.ID = table2.EMP_ID", conn)
outer_join_t2_to_t1

,EMP_ID,NAME,DEPARTMENT
0,1,Paul,IT Billing
1,2,Allen,Engineering
2,7,James,Finance


In [35]:
# union combines two select statements

union = pd.read_sql_query("SELECT EMP_ID, NAME, DEPARTMENT FROM table2 LEFT OUTER JOIN table1 ON table1.ID = table2.EMP_ID UNION SELECT EMP_ID, NAME, DEPARTMENT FROM table1 LEFT OUTER JOIN table2 ON table1.ID = table2.EMP_ID", conn)
union

,EMP_ID,NAME,DEPARTMENT
0,NaN,David,None
1,NaN,Kim,None
2,NaN,Mark,None
3,NaN,Teddy,None
4,1.0,Paul,IT Billing
5,2.0,Allen,Engineering
6,7.0,James,Finance


In [39]:
# union all simply appends two tables

union_all = pd.read_sql_query("SELECT EMP_ID, NAME, DEPARTMENT FROM table1 LEFT OUTER JOIN table2 ON table1.ID = table2.EMP_ID UNION ALL SELECT EMP_ID, NAME, DEPARTMENT FROM table2 LEFT OUTER JOIN table1 ON table1.ID = table2.EMP_ID", conn)
union_all

,EMP_ID,NAME,DEPARTMENT
0,1.0,Paul,IT Billing
1,2.0,Allen,Engineering
2,NaN,Teddy,None
3,NaN,Mark,None
4,NaN,David,None
5,NaN,Kim,None
6,7.0,James,Finance
7,1.0,Paul,IT Billing
8,2.0,Allen,Engineering
9,7.0,James,Finance
